In [2]:
from nustar_pysolar import planning
from nustar_pysolar.planning import _parse_timestamp as parse_timestamp
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import get_body, solar_system_ephemeris, get_body_barycentric, SkyCoord
from datetime import datetime

In [3]:
fname = planning.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_129.SHADOW_ANALYSIS.txt


In [4]:
tstart = '2017-05-19T01:43:00'
tend = '2017-05-19T11:43:00'

tstamp = '2017:137:09:15:00'
tstart = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                
tstamp = '2017:140:13:05:00'
tend = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                

orbits = planning.sunlight_periods(fname, tstart, tend)
solar_system_ephemeris.set('jpl')



<ScienceState solar_system_ephemeris: 'jpl'>

In [6]:
from skyfield.api import Loader
load = Loader('../data')

ts = load.timescale()
planets = load('jup310.bsp')


# Use the astropy interface to get the location of Juptier as the time that you want to use.

In [10]:
dt = 0.

# Using JPL Horizons web interface at 2017-05-19T01:34:40
horizon_ephem = SkyCoord(*[193.1535, -4.01689]*u.deg)

f = open('jupiter_20170517to20170520_pointing.txt', 'w')
last_ephem = None
for orbit in orbits:
    tstart = orbit[0]
    tend = orbit[1]
    print()
    on_time = (tend - tstart).total_seconds()
    
    point_time = tstart + 0.5*(tend - tstart)
    
    astro_time = Time(point_time)
    solar_system_ephemeris.set('jpl')


    jupiter = get_body('Jupiter', astro_time)
    
    jplephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    # Switch to the built in ephemris
    solar_system_ephemeris.set('builtin')
    jupiter = get_body('Jupiter', astro_time)
    
    builtin_ephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    t = ts.from_astropy(astro_time)
    jupiter, earth = planets['jupiter'], planets['earth']
    astrometric = earth.at(t).observe(jupiter)
    ra, dec, distance = astrometric.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem = SkyCoord(radeg, decdeg)
    
    if last_ephem is not None:
        print('Offset from last pointing: ', skyfield_ephem.separation(last_ephem))
    last_ephem=skyfield_ephem
    
    dt += on_time

    print()
    print(tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value))
    f.write(tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value)+'\n')
    
f.close()
    
print(dt)



2017-05-17T08:56:40 193.25724338413698  -4.0535299478020965

Offset from last pointing:  0d00m16.3945s

2017-05-17T10:33:20 193.25294336736857  -4.051999790568206

Offset from last pointing:  0d00m16.3512s

2017-05-17T12:10:00 193.24865442609843  -4.0504744953151635

Offset from last pointing:  0d00m16.3219s

2017-05-17T13:46:40 193.24437287127267  -4.048952750184888

Offset from last pointing:  0d00m16.2646s

2017-05-17T15:23:20 193.2401060744517  -4.047437174497219

Offset from last pointing:  0d00m16.2213s

2017-05-17T17:00:00 193.23585034187317  -4.045926454839385

Offset from last pointing:  0d00m16.178s

2017-05-17T18:36:40 193.23160567596435  -4.04442059240225

Offset from last pointing:  0d00m16.1347s

2017-05-17T20:13:20 193.22737208353217  -4.042919590454218

Offset from last pointing:  0d00m16.0914s

2017-05-17T21:50:00 193.22314957580977  -4.04142345430578

Offset from last pointing:  0d00m16.048s

2017-05-17T23:26:40 193.21893816847412  -4.039932191248995

Offset from la

In [12]:
planets.names()

{0: ['SOLAR_SYSTEM_BARYCENTER', 'SSB', 'SOLAR SYSTEM BARYCENTER'],
 3: ['EARTH_BARYCENTER',
  'EMB',
  'EARTH MOON BARYCENTER',
  'EARTH-MOON BARYCENTER',
  'EARTH BARYCENTER'],
 5: ['JUPITER_BARYCENTER', 'JUPITER BARYCENTER'],
 10: ['SUN'],
 399: ['EARTH'],
 501: ['IO'],
 502: ['EUROPA'],
 503: ['GANYMEDE'],
 504: ['CALLISTO'],
 505: ['AMALTHEA'],
 514: ['THEBE'],
 515: ['ADRASTEA'],
 516: ['METIS'],
 599: ['JUPITER']}